# Assignment 1
Student: Gabriele Grida 22-992-358

--- 
# IMPORTANT: all the submitted code should be in 2 cells
1) How you trained, evaluated and saved your model
2) How to load your model from a file, load the data and evaluate the model. Cell 2) should be running independently (even if cell 1 is not run)

<a href="https://colab.research.google.com/github/GiorgiaAuroraAdorni/ML-bachelor-course-assignments-sp23/blob/main/assignment%201/deliverable/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import io
import requests
import pickle
import numpy as np
import pandas as pd 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Load data 
url = 'https://drive.switch.ch/index.php/s/TeDwnbYsBKRuJjv/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# T1

# Split data into train and test set for x and y
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.85, shuffle=True, random_state=0)

print('train_data shape:', x_train.shape) # shape of the train sets
print('train_data shape:', y_train.shape)

print('test_data shape:', x_test.shape) # shape of the test sets
print('test_data shape:', y_test.shape)

# Define the model to predict the response variable
def linear_function(x, theta):
    x_1 = x[:,0]
    x_2 = x[:,1]
    y = theta[0] + theta[1] * x_1 + theta[2] * x_2 + theta[3] * np.sin(x_2) + theta[4] * x_1 * x_2
    return y

# Compact form
x_1 = x_train[:,0].reshape(-1,1)
x_2 = x_train[:,1].reshape(-1,1)
x_3 = np.sin(x_2)
x_4 = x_1 * x_2
ones_vector = np.ones(shape=(len(x_train),1))
X = np.hstack((ones_vector, x_1, x_2, x_3, x_4))         # The compact input
print('Input (X) shape:\t', X.shape)

# 1) Solve the linear system with np.linalg.solve()
theta_hat = np.linalg.solve(a=X.T.dot(X), b=X.T.dot(y_train))   # Solves ax = b with respect to x
print('Optimal parameters:\t', theta_hat)

# 2) Use LinearRegression
# init the model
lr = LinearRegression(fit_intercept=False)  

# estimate parameters
lr.fit(X, y_train)
theta_hat2 = lr.coef_

print('theta_hat = \t{}'.format(theta_hat))
print('theta_hat2 = \t{}'.format(theta_hat2))

# Evaluate the model's test performance
train_performance = mean_squared_error(linear_function(x_train,theta_hat), y_train)
test_performance = mean_squared_error(linear_function(x_test, theta_hat), y_test)

print("Train performance: \t", train_performance) # Mean squared error training set
print("Test performance: \t", test_performance) # Mean squared error test set

# save the model as a pickle file
with open('model_T1.pickle', 'wb') as file:
    pickle.dump(lr, file)

    
# T2

# Neural Networks
# Import libraries
import tensorflow as tf
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

# Set random seed
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Define the model
def create_model(neurons=10, activation="relu"):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X.shape[1],), activation=activation)) # input layer
    model.add(Dense(neurons, activation=activation)) # add a hidden layer with 10 neurons
    model.add(Dense(neurons, activation=activation)) # add a hidden layer with 10 neurons
    model.add(Dense(neurons, activation=activation)) # add a hidden layer with 10 neurons
    model.add(Dense(1, activation="linear")) # output layer
    model.compile(loss='mse', optimizer='sgd', metrics=['mse']) # stochastic gradient descent as optimizer
    return model

epochs = 200 # set epochs
model = create_model() # create the model

# Define the Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=0)

# train the model with 200 epochs and batch size of 32
history = model.fit(X, y_train, epochs=epochs, batch_size=32, verbose=0, validation_split=0.15, callbacks=[early_stop])

# evaluate the model on the training set
_,mse = model.evaluate(X, y_train)
print(f'MSE: {mse}')

# Use the test set to assess the performance
# Compact form
x_1 = x_test[:,0].reshape(-1,1)
x_2 = x_test[:,1].reshape(-1,1)
x_3 = np.sin(x_2)
x_4 = x_1 * x_2
ones_vector = np.ones(shape=(len(x_test),1))
X_test = np.hstack((ones_vector, x_1, x_2, x_3, x_4))         # compact input

# evaluate the model on the test set
_,mse = model.evaluate(X_test, y_test)
print(f'MSE test: {mse}')

# save the model as a pickle file
with open('model_T2.pickle', 'wb') as file:
    pickle.dump(model, file)

# T3 (Bonus)

# Set random seed
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Define the model
def create_model(neurons=14, activation="relu"):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(x_train.shape[1],), activation=activation)) # input layer
    model.add(Dense(neurons, activation=activation)) # add a hidden layer with 14 neurons
    model.add(Dense(neurons, activation=activation)) # add a hidden layer with 14 neurons
    model.add(Dense(neurons, activation=activation)) # add a hidden layer with 14 neurons
    model.add(Dense(1, activation="linear")) # output layer
    model.compile(loss='mse', optimizer='adam', metrics=['mse']) # Adam optimizer
    return model

epochs = 300 # set epochs
model_NN = create_model() # create the model

# Define the Early Stopping
early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=0)

# train the model with 300 epochs and batch size of 32
history = model_NN.fit(x_train, y_train, epochs=epochs, batch_size=32, verbose=0, validation_split=0.15, callbacks=[early_stop])

# evaluate the model on the test set
_,mse = model_NN.evaluate(x_test, y_test)
print(f'MSE test: {mse}')

# save the model as a pickle file
with open('model_T3.pickle', 'wb') as file:
    pickle.dump(model_NN, file)
    
    

train_data shape: (1700, 2)
train_data shape: (1700,)
test_data shape: (300, 2)
test_data shape: (300,)
Input (X) shape:	 (1700, 5)
Optimal parameters:	 [ 1.2731375  -0.0377395  -0.56711074  0.42078959  0.03463118]
theta_hat = 	[ 1.2731375  -0.0377395  -0.56711074  0.42078959  0.03463118]
theta_hat2 = 	[ 1.2731375  -0.0377395  -0.56711074  0.42078959  0.03463118]
Train performance: 	 0.7029378067715667
Test performance: 	 0.7685787172572301
54/54 [==============================] - 0s 3ms/step - loss: 0.1304 - mse: 0.1304
MSE: 0.1304306834936142
10/10 [==============================] - 0s 3ms/step - loss: 0.1165 - mse: 0.1165
MSE test: 0.11654753237962723
10/10 [==============================] - 0s 3ms/step - loss: 0.0210 - mse: 0.0210
MSE test: 0.02096232958137989


# Example on how to use baseline model:

In [2]:
# Import libraries
import joblib
import io
import requests
import numpy as np

def evaluate_predictions(y_true, y_pred):
    """
    Evaluates the mean squared error between the values in y_true and the values
    in y_pred.
    ### YOU CAN NOT EDIT THIS FUNCTION ###
    :param y_true: Numpy array, the true target values from the test set;
    :param y_pred: Numpy array, the values predicted by your model.
    :return: float, the mean squared error between the two arrays.
    """
    assert y_true.shape == y_pred.shape
    return ((y_true - y_pred) ** 2).mean()


def load_model(filename):
    """
    Loads a Scikit-learn model saved with joblib.dump.
    This is just an example, you can write your own function to load the model.
    Some examples can be found in src/utils.py.
    :param filename: string, path to the file storing the model.
    :return: the model.
    """
    model = joblib.load(filename)

    return model

# Load the data
# This will be replaced with our private test data when grading the assignment

# Load data from url
url = 'https://drive.switch.ch/index.php/s/TeDwnbYsBKRuJjv/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

# Alternatively yo can load the data from file
# data_path = '../data/data.npz'
# data = np.load(data_path)

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# Load the trained model
baseline_model_path = 'model_T1.pickle' # for T1
#baseline_model_path = 'model_T2.pickle' # for T2
#baseline_model_path = 'model_T3.pickle' # for T3
baseline_model = load_model(baseline_model_path)

# Change input
# Compact input for Task 1 and Task 2
x_1 = x[:,0].reshape(-1,1)
x_2 = x[:,1].reshape(-1,1)
x_3 = np.sin(x_2)
x_4 = x_1 * x_2
ones_vector = np.ones(shape=(len(x),1))
x = np.hstack((ones_vector, x_1, x_2, x_3, x_4))  

# input for Task 3
#x = data.f.x

# Predict on the given samples
y_pred = baseline_model.predict(x) 
y_pred = y_pred.reshape(-1)

############################################################################
# STOP EDITABLE SECTION: do not modify anything below this point.
############################################################################

# Evaluate the prediction using MSE
mse = evaluate_predictions(y_pred, y)
print(f'MSE on whole dataset: {mse}')

# NOTE: NOW THIS CELL IS NOT WORKING SINCE YOU NEED TO CHANGE THE INPUT.
# DO IT AND EVERYTHING RUNS SMOOTH


MSE on whole dataset: 0.7127839433444162
